# GHS Data Download and Aggregating

This notebook describes the process of downloading and aggregating GHS data from [here](https://ghsl.jrc.ec.europa.eu/download.php?). The notebook contains aggregation for the grid:

- Classification of pixels as rural or urban for the raster.
    - Using 21 and above for urban and 13 and below for rural.
- Fraction of each grid pixel that is rural/ urban
- Find population by grid pixel.

The epoch used in closest to the date `27th Feb 2023` and would be 2025.

In [101]:
%load_ext jupyter_black
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os
import requests, zipfile, io
import rasterio
import rioxarray as rxr

from rasterstats import zonal_stats

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [102]:
base_dir = Path(os.getenv("STORM_DATA_DIR")) / "analysis/02_new_model_input/"
input_dir = base_dir / "06_settlement/input/"
shp_input_dir = base_dir / "02_housing_damage/input/"
grid_input_dir = base_dir / "02_housing_damage/output/"
output_dir = base_dir / "06_settlement/output/"

In [103]:
adm3_shp = gpd.read_file(
    shp_input_dir / "phl_adminboundaries_candidate_adm3.zip"
)

# grid
grid = gpd.read_file(grid_input_dir / "phl_0.1_degree_grid_land_overlap.gpkg")

In [104]:
smod_link = "https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_SMOD_GLOBE_R2022A/GHS_SMOD_P2025_GLOBE_R2022A_54009_1000/V1-0/GHS_SMOD_P2025_GLOBE_R2022A_54009_1000_V1_0.zip"
pop_link = "https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_POP_GLOBE_R2022A/GHS_POP_P2025_GLOBE_R2022A_54009_100/V1-0/tiles/"

## Degree of Urbanisation

### Urban-Rural Classification

In [105]:
# Downloading the whole global data set as it is small
req = requests.get(smod_link, verify=False, stream=True)
with zipfile.ZipFile(io.BytesIO(req.content)) as zObj:
    fileNames = zObj.namelist()
    for fileName in fileNames:
        if fileName.endswith("tif"):
            content = zObj.open(fileName).read()
            open(input_dir / "SMOD" / fileName, "wb").write(content)

c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\globaltyphoon\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jeodpp.jrc.ec.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [106]:
# Reading in raster
file_name = os.listdir(input_dir / "SMOD")
smod_raster = rasterio.open(input_dir / "SMOD" / file_name[0])
smod_array = smod_raster.read(1)
smod_array

array([[-200, -200, -200, ..., -200, -200, -200],
       [-200, -200, -200, ..., -200, -200, -200],
       [-200, -200, -200, ..., -200, -200, -200],
       ...,
       [-200, -200, -200, ..., -200, -200, -200],
       [-200, -200, -200, ..., -200, -200, -200],
       [-200, -200, -200, ..., -200, -200, -200]], dtype=int16)

In [107]:
# no data are set at -200
# water seems to be set to 10
# converting to similar CRS
smod_raster.crs

CRS.from_wkt('PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [108]:
grid.total_bounds

array([114.25,   4.55, 126.65,  21.15])

In [109]:
# converting crs and clipping
# checking if crs are the same
smod_raster.crs == grid.crs

False

In [110]:
smod_raster = rxr.open_rasterio(input_dir / "SMOD" / file_name[0])
smod_raster.rio.crs

CRS.from_wkt('PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [111]:
smod_raster_wgs84 = smod_raster.rio.reproject(grid.crs)
smod_raster_wgs84_clip = smod_raster_wgs84.rio.clip_box(*grid.total_bounds)

In [112]:
smod_grid_vals = pd.DataFrame(
    {
        "id": grid["id"],
        "Centroid": grid["Centroid"],
        "urban": None,
        "rural": None,
        "water": None,
    }
)
for grd in grid.Centroid:
    grd_sel = grid[grid.Centroid == grd]
    grid_rast = smod_raster_wgs84_clip.rio.clip(
        grd_sel["geometry"], all_touched=True
    )
    smod_grid_vals.loc[grd_sel.index.values, ["urban"]] = (
        (grid_rast >= 21) & (grid_rast <= 30)
    ).sum().values / grid_rast.count().values
    smod_grid_vals.loc[grd_sel.index.values, ["rural"]] = (
        (grid_rast >= 11) & (grid_rast <= 13)
    ).sum().values / grid_rast.count().values
    smod_grid_vals.loc[grd_sel.index.values, ["water"]] = (
        grid_rast == 10
    ).sum().values / grid_rast.count().values

In [113]:
smod_grid_vals.tail(10)

,id,Centroid,urban,rural,water
3716,20513,126.5E_7.3N,0.008264,0.991736,0.0
3717,20514,126.5E_7.2N,0.049587,0.85124,0.099174
3718,20515,126.5E_7.1N,0.066116,0.363636,0.570248
3719,20516,126.5E_7.0N,0.041322,0.024793,0.933884
3720,20676,126.6E_7.7N,0.024793,0.099174,0.876033
3721,20677,126.6E_7.6N,0.07438,0.181818,0.743802
3722,20678,126.6E_7.5N,0.0,0.479339,0.520661
3723,20679,126.6E_7.4N,0.0,0.190083,0.809917
3724,20680,126.6E_7.3N,0.033058,0.297521,0.669421
3725,20681,126.6E_7.2N,0.024793,0.099174,0.876033


In [114]:
del (
    smod_raster,
    smod_array,
    smod_raster_wgs84,
    smod_raster_wgs84_clip,
)

## Population

### Total Population by grid

In [115]:
# downloading the popoulation data
# selected from here: https://ghsl.jrc.ec.europa.eu/download.php?ds=pop
phl_boxes = ["R7_C30", "R7_C31", "R8_C30", "R8_C31", "R9_C30", "R9_C31"]
file_list = [
    "GHS_POP_P2025_GLOBE_R2022A_54009_100_V1_0_" + patt + ".zip"
    for patt in phl_boxes
]

In [116]:
for file in file_list:
    req = requests.get(pop_link + file, allow_redirects=True)
    with zipfile.ZipFile(io.BytesIO(req.content)) as zObj:
        fileNames = zObj.namelist()
        for fileName in fileNames:
            if fileName.endswith("tif"):
                content = zObj.open(fileName).read()
                open(input_dir / "POP" / fileName, "wb").write(content)

This section took long to process.

NOTE: 
- The raster layers and the grid do not have the same CRS.
    - Options are to project grid to raster CRS or raster to grid CRS.
    - Projecting to grid CRS seems to produce realistic numbers.
    - Numbers still seem slightly distorted based on visual inspection on QGIS.
    - Example: grid `117.0E_8.2N` 
        - shows a sum of 197.34 visually in QGIS
        - shows a sum of 197.34 after re-projecting the grid to raster CRS.
        - shows a sum of 201.343014 when re-projecting the rasters individually.
        - shows a sum of 355.782260 when merging and re-projecting
        - re-projecting the grid seems the way to go.
- Merging raster layers into one takes a lot of storage. 
    - It also results in higher values.
- A re-projected raster layer takes even more space and is better to just re-project on the fly.

In [117]:
# merging rasters using gdal
#! gdalbuildvrt "%STORM_DATA_DIR%/analysis/02_new_model_input/06_settlement/input/POP/PHL_GHS_POP_P2025_R2022A_54009_100_V1_0.vrt" "%STORM_DATA_DIR%/analysis/02_new_model_input/06_settlement/input/POP/GHS*.tif"
# translating virtual mosaic to geotiff
#! gdal_translate -of GTiff -co "TILED=YES" "%STORM_DATA_DIR%/analysis/02_new_model_input/06_settlement/input/POP/PHL_GHS_POP_P2025_R2022A_54009_100_V1_0.vrt" "%STORM_DATA_DIR%/analysis/02_new_model_input/06_settlement/input/POP/PHL_GHS_POP_P2025_R2022A_54009_100_V1_0.tif"

In [118]:
# opening files and merging them
pop_grid_vals = pd.DataFrame(
    {
        "id": grid["id"],
        "Centroid": grid["Centroid"],
        "total_pop": None,
    }
)

In [119]:
## SOLUTION: convert grid to raster CRS

file_list = os.listdir(input_dir / "POP")
tif_list = [tif for tif in file_list if tif.endswith(".tif")]
pop_raster = rasterio.open(input_dir / "POP" / tif_list[0])
grid_crs = grid.to_crs(pop_raster.crs.to_dict())

In [120]:
for file in tif_list:
    pop_raster = rasterio.open(input_dir / "POP" / file)
    pop_array = pop_raster.read(1)
    pop_stats = zonal_stats(
        grid_crs,
        pop_array,
        stats=["sum"],
        nodata=-200,
        all_touched=True,
        affine=pop_raster.transform,
    )
    grid_stats = pd.DataFrame(pop_stats)
    pop_grid_vals[phl_boxes[tif_list.index(file)]] = grid_stats["sum"]

In [121]:
# takes too long
# tif_list = os.listdir(input_dir / "POP")
# for file in tif_list[2:3]:
#    pop_rast = (
#        rxr.open_rasterio(input_dir / "POP" / file)
#        # .rio.reproject(grid.crs)
#        # .rio.clip_box(*grid.total_bounds)
#    )
#    for grd in grid_crs.Centroid[0:10]:
#        grd_sel = grid_crs[grid_crs.Centroid == grd]
#        try:
#            grid_rast = pop_rast.rio.clip(
#                grd_sel["geometry"], all_touched=True
#            )
#            pop_grid_vals.loc[
#                grd_sel.index.values, [phl_boxes[tif_list.index(file)]]
#            ] = ((grid_rast.where(grid_rast >= 0)).sum().values)
#        except:
#            pop_grid_vals.loc[
#                grd_sel.index.values, [phl_boxes[tif_list.index(file)]]
#            ] = 0

In [122]:
# sum all columns
pop_grid_vals["total_pop"] = pop_grid_vals.loc[:, phl_boxes].sum(axis=1)

In [123]:
pop_grid_vals

,id,Centroid,total_pop,R7_C30,R7_C31,R8_C30,R8_C31,R9_C30,R9_C31
0,101,114.3E_11.1N,0.000000,NaN,None,NaN,NaN,NaN,NaN
1,4475,116.9E_7.9N,0.000000,NaN,None,NaN,NaN,0.000000,NaN
2,4639,117.0E_8.2N,197.339034,NaN,None,197.339034,NaN,NaN,NaN
3,4640,117.0E_8.1N,5118.586441,NaN,None,3991.248616,NaN,1127.337824,NaN
4,4641,117.0E_8.0N,12421.360279,NaN,None,NaN,NaN,12421.360279,NaN
...,...,...,...,...,...,...,...,...,...
3721,20677,126.6E_7.6N,17758.051774,NaN,None,NaN,NaN,NaN,17758.051774
3722,20678,126.6E_7.5N,6383.934604,NaN,None,NaN,NaN,NaN,6383.934604
3723,20679,126.6E_7.4N,6933.484614,NaN,None,NaN,NaN,NaN,6933.484614
3724,20680,126.6E_7.3N,11363.645484,NaN,None,NaN,NaN,NaN,11363.645484


In [124]:
pop_grid_vals["total_pop"].sum()

119328366.48788711

In [125]:
# merging the two dataframes
merged_ghs_df = smod_grid_vals.merge(
    pop_grid_vals[["id", "Centroid", "total_pop"]], on=["id", "Centroid"]
)

In [126]:
merged_ghs_df

,id,Centroid,urban,rural,water,total_pop
0,101,114.3E_11.1N,0.0,0.0,1.0,0.000000
1,4475,116.9E_7.9N,0.0,0.024793,0.975207,0.000000
2,4639,117.0E_8.2N,0.0,0.008264,0.991736,197.339034
3,4640,117.0E_8.1N,0.0,0.338843,0.661157,5118.586441
4,4641,117.0E_8.0N,0.0,0.793388,0.206612,12421.360279
...,...,...,...,...,...,...
3721,20677,126.6E_7.6N,0.07438,0.181818,0.743802,17758.051774
3722,20678,126.6E_7.5N,0.0,0.479339,0.520661,6383.934604
3723,20679,126.6E_7.4N,0.0,0.190083,0.809917,6933.484614
3724,20680,126.6E_7.3N,0.033058,0.297521,0.669421,11363.645484


In [127]:
# writing output
merged_ghs_df.to_csv(output_dir / "ghs_rural_urban_pop.csv", index=False)